# **Background**
## **Knowledge Graph Evaluation and Metrics**
This notebook seeks to identify the different points of evaluation for developing knowledege graphs, in order to establish metrics for evaluating response performance. Below lists the three different stages of RAG where each stage identifies specific control points for adjusting configuration and input parameters for improving RAG performance.

## **Neo4j and Three Stages of GraphRAG**
Neo4j is a graph database management system that is designed to handle large amounts of data in the form of graphs. Unlike traditional relational databases, Neo4j utilizes graph structures consisting of nodes, edges (relationships), and properties. Cypher Query language is utilized for retrieving the relevant nodes from a graph based on the modes and initial configurations set for the retrieval process. 

The three stages of GraphRAG are:
1. Ingestion and Preprocessing
2. Retrieval
3. Response

Each stage have various input and configuration parameters that can affect RAG performance and its metrics. 

The functions are based on the public [Neo4j llm-graph-builder repo](!https://github.com/neo4j-labs/llm-graph-builder)

### **Quick Guide**
If you're looking to test various queries and see how different modes of retrievals outputs a response, then follow the following steps:

1. Change the ```question``` to test different queries
2. Restart Kennel and Run all
3. Look at the **Comparisons** cell for response results (2nd to last cell)

*References*: 
- [Neo4j llm-graph-builder Github repo](https://github.com/neo4j-labs/llm-graph-builder)
- [RAG TRIAD Metrics](https://truera.com/ai-quality-education/generative-ai-rags/what-is-the-rag-triad/)



## **Initial Setup**

In [1]:
# Add path to sys.path to enable methods within backend/src (do at least once)
import os
import sys
import logging
sys.path.append('/home/shinhojung/llm-graph-builder/backend')

from dotenv import load_dotenv
load_dotenv("example.env")

# print(sys.path)
# print(os.getenv("OPENAI_API_KEY"))

Python-dotenv could not parse statement starting at line 32


True

### **Ingestion and Preprocessing**
The ingestion method for this demo utilizes the [Neo4j App](https://llm-graph-builder.neo4jlabs.com/) for uploading and processing the unstructured data (i.e PDF), which is outside of this Juypter Notebook. Through the Neo4j App, data is chunked into smaller, managable pieces with overlapping, predefined fixed-length parameters. Then each chunk is processed through an LLM (i.e chatGPT, gemini) under its Named Entity Recognition (NER) capability to create nodal entities and their associated relationships. Once these entities are created, an embedding model creates vector embeddings of each node, capturing semantic meaning of the entity and its surrounding text, and then stores these vector embeddings as a property of the respective nodes.

**NOTE**: All chunk nodes are created and connected to an associated document node.

**Levers of Control:**
- **Schema & Chunk Tailoring**
  - Predefined schema can help identiify specific entities and relationships that are more suitable and oriented to the targeted domain or application, affecting the overall RAG graph search and document retrieval methods.
  - Neo4j has a default schema during ingestion and preprocessing.
- **Embedding Model Assessment**
  - Different emedding models (i.e all-MiniLM-L6-v2) provide different numeric values in vector embeddings affecting similarity search results.

## **Connecting with a Neo4j DB instance**
Run the backend of the llm-graph-builder repo by running the following command within the ```llm-graph-builder/backend``` folder:

```uvicorn score:app --reload --log-level debug```

This runs a local server that enables API defined by the ```llm-graph-builder``` public repo. 

**NOTE**: This assumes there's an existing Neo4j instance already created with a graph using the Neo4j app.

In [2]:
# Connect with Neo4j Database @/connect
import requests
import json

# Define Neo4j Database connection

## Anatomy & Physiology
uri="neo4j+s://819c2e86.databases.neo4j.io:7687"
userName="neo4j"
password="3rLOPhwRVsZa6zzv7Nl0EusFB2Rocl_OTH34UIUeayw"
database="neo4j"

# # Compliance knowledge Graph
# uri="neo4j+s://db5d18b3.databases.neo4j.io:7687"
# userName="neo4j"
# password="MHCzaBJ3OPoDIKeJyuxVWNiWqGUVmB5s5BEHUw4KvCA"
# database="neo4j"


url = "http://127.0.0.1:8000/connect"

data = {
    "uri": uri,
    "userName": userName,
    "password":password,
    "database":database
}

content_length = len(data)
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Content-Length":"4"
}
json_data = json.dumps(data)

response = requests.post(url, headers=headers, data=data)

print(response.text)

{"status":"Success","data":{"db_vector_dimension":384,"application_dimension":384,"message":"Connection Successful"}}


## **Visualize Graph in Juypter Notebooks using yfiles**
Compare the yfiles widget graph display with Neo4j's online [visualization tool](!https://llm-graph-builder.neo4jlabs.com/)

In [3]:
# Visualize Graph
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

default_cypher = "MATCH (n:Document)<-[r]->(c:Chunk)<-[s]->(e) RETURN n,r,c,s,e LIMIT 100"

def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri = uri,
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

### **Find number of nodes and relationships in Knowledge Graph**

In [4]:
# /graph_query >> get_graph_results function
from src.graph_query import get_graphDB_driver, execute_query,extract_node_elements,extract_relationships
from src.shared.constants import GRAPH_QUERY, GRAPH_CHUNK_LIMIT

driver = get_graphDB_driver(uri,userName,password)
query = GRAPH_QUERY.format(graph_chunk_limit=GRAPH_CHUNK_LIMIT)
# List documents filenames into document_names. NOTE: By default method should work while empty but does not for some unknown reason. 
document_names =["The Central Nervous System _ Anatomy and Physiology I.pdf","The Embryologic Perspective _ Anatomy and Physiology I.pdf","Circulation and the Central Nervous System _ Anatomy and Physiology I.pdf","The Peripheral Nervous System _ Anatomy and Physiology I.pdf","Glossary_ The Nervous System _ Anatomy and Physiology I.pdf","Introduction to the Nervous System _ Anatomy and Physiology I.pdf"]
records, summary, keys = execute_query(driver, query, document_names, doc_limit=None)
document_nodes = extract_node_elements(records)
document_relationships = extract_relationships(records)

# print(f"records: {records}")
# print(f"summary: {summary}")
# print(f"keys: {keys}")

print(f"no of nodes : {len(document_nodes)}")
print(f"no of relations : {len(document_relationships)}")
# results = {
#     "nodes": document_nodes,
#     "relationships": document_relationships
# }


no of nodes : 727
no of relations : 3084


## **Query the Existing Knowledge Graph**
The Neo4j Cypher query ``` MATCH (d:Document) Return d``` shows all nodes labeled as "Document".
Within the records, look for the number of chunks.

**Example**
```
records: [<Record d=<Node element_id='4:187c0626-2398-4709-a30d-c929bee00f7d:2' 
labels=frozenset({'Document'}) 
properties={
    'fileName': 'Anatomy_and_Physiology_CH13.pdf', 
    'errorMessage': '', 
    'fileSource': 'local file', 
    'total_chunks': 188, 
    'processingTime': 435.63, 
    'createdAt': neo4j.time.DateTime(2024, 10, 13, 23, 10, 11, 139430000), 
    'fileSize': 13380426, 'nodeCount': 799, 
    'model': 'openai-gpt-4o', 
    'processed_chunk': 188, 
    'fileType': 'pdf', 
    'relationshipCount': 532, 
    'is_cancelled': False, 
    'status': 'Completed', 
    'updatedAt': neo4j.time.DateTime(2024, 10, 13, 23, 17, 39, 134987000)}>>]```


In [5]:
# /graph_query >> Query test
from src.graph_query import get_graphDB_driver, execute_query,extract_node_elements,extract_relationships

driver = get_graphDB_driver(uri,userName,password)
query =  """
MATCH (d:Document)
RETURN d

"""  # labels are case-sensitive
records, summary, keys = execute_query(driver, query, document_names, doc_limit=None)
document_nodes = extract_node_elements(records)
document_relationships = extract_relationships(records)

print(f"records: {records}")
# print(f"summary: {summary}")
# print(f"keys: {keys}")
# print(f"document_nodes: {document_nodes}")
# print(f"document_relationships: {document_relationships}")

records: [<Record d=<Node element_id='4:187c0626-2398-4709-a30d-c929bee00f7d:0' labels=frozenset({'Document'}) properties={'fileName': 'Glossary_ The Nervous System _ Anatomy and Physiology I.pdf', 'errorMessage': '', 'fileSource': 'local file', 'total_chunks': 35, 'processingTime': 141.66, 'createdAt': neo4j.time.DateTime(2024, 10, 22, 18, 57, 18, 300405000), 'fileSize': 384987, 'nodeCount': 174, 'model': 'openai-gpt-4o', 'processed_chunk': 35, 'fileType': 'pdf', 'relationshipCount': 0, 'is_cancelled': False, 'status': 'Completed', 'updatedAt': neo4j.time.DateTime(2024, 10, 22, 18, 59, 51, 989990000)}>>, <Record d=<Node element_id='4:187c0626-2398-4709-a30d-c929bee00f7d:1' labels=frozenset({'Document'}) properties={'fileName': 'The Embryologic Perspective _ Anatomy and Physiology I.pdf', 'errorMessage': '', 'fileSource': 'local file', 'total_chunks': 30, 'processingTime': 134.16, 'createdAt': neo4j.time.DateTime(2024, 10, 22, 18, 57, 18, 909662000), 'fileSize': 783572, 'nodeCount': 85

### **Retrieval**
Given a Neo4j graph-vector store database, the performance of document retrieval is determined the quality of translation of the initial user query and execution of the cypher query with the Neo4j database.  Traditional data science metrics (i.e precision, recall, F1 score) are used to measure the amount of relevant documents the GraphQARetriever can obtain. Different modes of retrieval (i.e graph-only, vector-only, hybrid (vector and keyword) search, and hybrid-cypher-query search were explored within this Juypter Notebook.

**Levers of Control**
- **Query to Cypher Statement**
  - Converting the user query to relevant cypher statement using llm (i.e Gemini, OpenAI) prior to creating a document retriever object.
- **Query to Embedding**
  - Converting the user query to text embedding for vector similarity search (default cosine similarity)
- **Document Retrieval Assessment**
  - Retrieving the relevant documents from the Graph/Vector DB (i.e GraphQARetriever) using various graph traversal and vector similarity serach techniques


In [6]:
# Deconstructed: /chat_bot API >> QA_RAG >> create_graph_chain
from src.llm import get_llm
import langchain_community.graphs.neo4j_graph as n
from langchain.chains import GraphCypherQAChain

# Fetching graph from database using user agent
graph = n.Neo4jGraph(url=uri,username=userName,password=password,database=database,sanitize = True, refresh_schema=True, driver_config={'user_agent':os.environ.get('NEO4J_USER_AGENT')})

# RAG Parameters
model="gemini-1.5-flash-001"
question="What are all the arteries in the Circle of Willis"
session_id = None

# Create graph chain
cypher_llm,model_name = get_llm(model)
qa_llm,model_name = get_llm(model)
graph_chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm,
    qa_llm=qa_llm,
    validate_cypher= True,
    graph=graph,
    # verbose=True, 
    return_intermediate_steps = True,
    top_k=3
)

# print(graph_chain)
# print(qa_llm)

# Deconstructed: /chat_bot API >> QA_RAG >> get_graph_response
# Graph Retrieval
from src.QA_integration_new import get_graph_response
graph_response = get_graph_response(graph_chain,question)
print("This is graph response:")
print(graph_response)



/home/shinhojung/llm-graph-builder/backend/src/shared/common_fn.py:89: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = SentenceTransformerEmbeddings(
/home/shinhojung/llm-graph-builder/backend/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


This is graph response:
{'response': "I don't know the answer. \n", 'cypher_query': 'MATCH (a:Anatomical_structure {description: "Circle of Willis"})<-[:PART_OF]-(b:Anatomical_structure) WHERE b.description CONTAINS "artery" RETURN b.description', 'context': []}


### **Response**
After retrieving the relevant documents, the llm-graph-builder aggregations the chunks in sequential order while filtering out less relevant infromation. Then, the query response summarized

**Levers of Control**
- **Embedding to Response**
  - Converting the embedding of the retrieved documents to human readable response (using LLM i.e Gemini)
- **Human and Validation Assessment**
  - Ensuring if the response answers the original query.


In [7]:
from src.QA_integration_new import load_embedding_model
# Identify the embedding model
EMBEDDING_MODEL = os.getenv('EMBEDDING_MODEL')
EMBEDDING_FUNCTION , _ = load_embedding_model(EMBEDDING_MODEL)
# Identify the LLM used
llm,model_name = get_llm(model)

print(EMBEDDING_FUNCTION)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [8]:
# Create messages needed to prompt LLM to retreive human readable response text
from src.QA_integration_new import create_neo4j_chat_message_history
from langchain_core.messages import HumanMessage

history = create_neo4j_chat_message_history(graph, session_id=" ")
messages = history.messages
user_question = HumanMessage(content=question)
messages.append(user_question)

print(messages)

[HumanMessage(content='What are all the arteries in the Circle of Willis')]


### **Vector Search**

- Performs primarily based on vector similarity between query embedding and embedding property of nodes within the knowledge graph.
- Though primarily utilizes vector index search, the ```VECTOR_SEARCH_QUERY``` also focuses soley on ```PART_OF``` relationships, where a chunk is a part of document, ignoring any other relationships or entitites in the graph.
- Does not perform any other specific entity-based matching or similarity checks (more suitable for more starightforward document retrieval based soley on chunk scores)

#### **Scoring**
- The embedding of each chunk node (i.e a text segment) is evaluated for similarity with the input query embedding.
- For each document, the scores of all chunks are averaged via 'avg(score)' to generate a single socre representing the document. This average score gives an overall measure of how relevant the coument is to the query.
- **Return Value**: This average score is returned as the score for each document, alongside the concatenated text from its chunks and relevant metadata. 

In [9]:
# Deconstructed: /chat_bot API >> QA_RAG >> VECTOR_GRAPH_SEARCH and setup_chat
# Different from RAG_Graph
from src.shared.constants import VECTOR_SEARCH_QUERY
from src.QA_integration_new import retrieve_documents, process_documents, create_document_retriever_chain
from src.shared.constants import CHAT_SEARCH_KWARG_K, CHAT_SEARCH_KWARG_SCORE_THRESHOLD
from src.QA_integration_new import format_documents, get_rag_chain, get_sources_and_chunks, get_total_tokens
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

def vector_retrieval():
    mode = "vector"

    # under QA_RAG with Hybrid Search
    retrieval_query = VECTOR_SEARCH_QUERY

    # Create retriever
    index_name = "vector"  # default
    score_threshold=CHAT_SEARCH_KWARG_SCORE_THRESHOLD

    # Vectors within a graph (aka. setup chat)
    neo_db = Neo4jVector.from_existing_index(
                    embedding=EMBEDDING_FUNCTION,
                    index_name=index_name,
                    retrieval_query=retrieval_query,
                    graph=graph
                )
    retriever = neo_db.as_retriever(search_type="similarity_score_threshold",search_kwargs={"score_threshold": score_threshold}) # default k = 4 not 3
    doc_retriever = create_document_retriever_chain(llm, retriever)
    return doc_retriever, mode

doc_retriever, mode = vector_retrieval()
docs = retrieve_documents(doc_retriever, messages)

if docs:
    formatted_docs, sources = format_documents(docs,model)
    rag_chain = get_rag_chain(llm=llm)
    ai_response = rag_chain.invoke({
        "messages": messages,
        "context": formatted_docs,
        "input": question 
    })
    result = get_sources_and_chunks(sources, docs)
    content = ai_response.content
    total_tokens = get_total_tokens(ai_response,llm)
else:
    content = "I couldn't find any relevant documents to answer your question."
    result = {"sources": [], "chunkdetails": []}
    total_tokens = 0

vector_content = content
vector_result = result
vector_total_tokens = total_tokens
print()
print(f"message: {vector_content}")
print(f"sources: {vector_result['sources']}")
# print(f"model: {model_version}")
print(f"chunkdetails: {vector_result['chunkdetails']}")
print(f"total_tokens: {vector_total_tokens}")
print(f"mode: {mode}")






message: The Circle of Willis is a ring-shaped structure at the base of the brain that ensures continuous blood flow to the brain. It is formed by the following arteries:

* **Internal carotid arteries:** These arteries branch from the common carotid arteries and enter the skull through the carotid canal.
* **Anterior cerebral arteries:** These arteries arise from the internal carotid arteries and supply blood to the frontal lobes and parts of the parietal lobes.
* **Anterior communicating artery:** This small artery connects the two anterior cerebral arteries.
* **Posterior cerebral arteries:** These arteries arise from the basilar artery and supply blood to the occipital lobes and parts of the temporal lobes.
* **Posterior communicating arteries:** These arteries connect the internal carotid arteries to the posterior cerebral arteries.
* **Basilar artery:** This artery is formed by the merger of the two vertebral arteries and supplies blood to the brainstem and cerebellum. 

sources

### **Hybrid (Vector+Keyword) Search**
- Utilizes both vector and full-text index to query relevant documents.
- Incorporates ```HAS_ENTITY``` relationships and allows the query to handle both embedded entities and multiple levels of relationships
- Uses cosine similarity checks between chunk embeddings and entity embeddings to filter entities based on configurable similarity thresholds (embedding_match_min and embedding_match_max)
- Allows for recurisve querying to a configurable depth, capturing paths to connected entities and allowing the search to pull in more contextually relevant nodes
- Suitable for applications that require an understanding of semantic connections between entities within a graph, such as knowledge graph exploartion, entity-based search, or contextual entity disambiguation

#### **Scoring**
- **Score Collection and Averaging**: Similar to ```VECTOR_SEARCH_QUERY```, each chunk has an initial score, and for each document, the scores of chunks are averaged to produce ```avg_score```
- **Entity Filtering with Embeddings**: Entities connected to each ```chunk``` are included based on embedding similarity threshold, using cosine similarity.
  - Enitties without embeddings are directly included
  - Entities with embeddings are included if their cosine similarity to the query embedding falls within the specified range (```embedding_match_min``` and ```embedding_match_max```).
  - If an entity's similarity exceeds the ```embedding_match_max```, broader relationships are included in the entity's graph.
- **Return Value**: Like ```VECTOR_SEARCH_QUERY```, the ```avg_score``` is returned as the main score; however, this score now incorporates a more complex structure of entities and relationships relevant to the query.


In [10]:
# Deconstructed: /chat_bot API >> QA_RAG >> VECTOR_GRAPH_SEARCH and setup_chat
# Different from RAG_Graph
from src.shared.constants import VECTOR_GRAPH_SEARCH_QUERY, VECTOR_GRAPH_SEARCH_ENTITY_LIMIT, VECTOR_SEARCH_QUERY
from src.QA_integration_new import retrieve_documents, process_documents, create_document_retriever_chain
from src.shared.constants import CHAT_SEARCH_KWARG_K, CHAT_SEARCH_KWARG_SCORE_THRESHOLD
from langchain_community.vectorstores.neo4j_vector import Neo4jVector


def hybrid_retrieval():
    mode ="graph + vector + fulltext"

    # under QA_RAG with Hybrid Search
    retrieval_query = VECTOR_GRAPH_SEARCH_QUERY.format(no_of_entites=VECTOR_GRAPH_SEARCH_ENTITY_LIMIT)

    # Create retriever
    index_name = "vector"
    keyword_index = "keyword"
    search_k=CHAT_SEARCH_KWARG_K, 
    score_threshold=CHAT_SEARCH_KWARG_SCORE_THRESHOLD

    # Vectors within a graph (aka. setup chat)
    neo_db = Neo4jVector.from_existing_graph(
                    embedding=EMBEDDING_FUNCTION,
                    index_name=index_name,
                    retrieval_query=retrieval_query,
                    graph=graph,
                    search_type="hybrid",
                    node_label="Chunk",
                    embedding_node_property="embedding",
                    text_node_properties=["text"],
                    keyword_index_name=keyword_index
                    )
    retriever = neo_db.as_retriever(search_type="similarity_score_threshold",search_kwargs={"score_threshold": score_threshold}) # default k = 4 not 3
    doc_retriever = create_document_retriever_chain(llm, retriever)
    return doc_retriever, mode

# Initiate the retrieval
doc_retriever, mode = hybrid_retrieval()
docs = retrieve_documents(doc_retriever, messages)

if docs:
    content, result, total_tokens, predict_time = process_documents(docs, question, messages, llm,model)
else:
    content = "I couldn't find any relevant documents to answer your question."
    result = {"sources": [], "chunkdetails": []}
    total_tokens = 0


hybrid_content = content
hybrid_result = result
hybrid_total_tokens = total_tokens

print()
print(f"message: {hybrid_content}")
print(f"sources: {hybrid_result['sources']}")
# print(f"model: {model_version}")
print(f"chunkdetails: {hybrid_result['chunkdetails']}")
print(f"total_tokens: {hybrid_total_tokens}")
print(f"mode: {mode}")
print(f"predict_time: {predict_time}")



message: The Circle of Willis is a network of arteries at the base of the brain. The arteries that contribute to the Circle of Willis are:

* **Internal carotid arteries:** These arteries enter the cranium through the carotid canal and branch into the anterior cerebral artery and the middle cerebral artery.
* **Vertebral arteries:** These arteries merge to form the basilar artery, which then branches into the posterior cerebral arteries. 

The Circle of Willis is crucial for maintaining blood flow to the brain, even if there is a blockage in one of the arteries. 

sources: {'Circulation and the Central Nervous System _ Anatomy and Physiology I.pdf', 'Glossary_ The Nervous System _ Anatomy and Physiology I.pdf'}
chunkdetails: [{'id': 'df99f8192bf2961d93f8fa02eb32337088dbeffb', 'score': 1.0}, {'id': 'f73ae203cb7d3238826460d93b4dcbef24801c62', 'score': 0.9588}, {'id': '8eee3b357209bc21f61d633156b9d00b829ddc73', 'score': 0.8606}, {'id': '24564789b83b665072530bcf37943ede49aa1201', 'score':

### **Hybrid Cypher Retriever**
- Conducts another cypher search on top of hybrid (vector+keyword) search

In [11]:
# Deconstructed: /chat_bot API >> QA_RAG >> VECTOR_GRAPH_SEARCH and setup_chat
# Different from RAG_Graph
from src.shared.constants import VECTOR_GRAPH_SEARCH_QUERY, VECTOR_GRAPH_SEARCH_ENTITY_LIMIT, VECTOR_SEARCH_QUERY
from src.QA_integration_new import retrieve_documents, process_documents, create_document_retriever_chain
from src.shared.constants import CHAT_SEARCH_KWARG_K, CHAT_SEARCH_KWARG_SCORE_THRESHOLD
from src.llm import get_llm
from neo4j_graphrag.retrievers import HybridCypherRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import VertexAILLM

mode ="graph + vector + fulltext"

# under QA_RAG with Hybrid Search
retrieval_query = VECTOR_GRAPH_SEARCH_QUERY.format(no_of_entites=VECTOR_GRAPH_SEARCH_ENTITY_LIMIT)

# Create retriever
index_name = "vector"
keyword_index = "keyword"

retriever = HybridCypherRetriever(
    driver=driver,
    vector_index_name=index_name,
    fulltext_index_name=keyword_index,
    retrieval_query=retrieval_query,
    embedder=EMBEDDING_FUNCTION,
)

llm = VertexAILLM(model_name="gemini-1.5-flash-001")
rag = GraphRAG(retriever=retriever, llm=llm)

retriever_result = retriever.search(query_text=question, top_k=4)
hybridcypher_response = rag.search(query_text=question,retriever_config={"top_k":4})
print()
print(hybridcypher_response.answer)
print()
hybridcypher_chunkdetails = []
for m in retriever_result.items:
    hybridcypher_chunkdetails.append(m.metadata["chunkdetails"])


The arteries that contribute to the Circle of Willis are:

* **Internal Carotid Arteries** 
* **Vertebral Arteries** 
* **Basilar Artery** 




## **Compare Graph, Vector, Hybrid (Vector+Keyword), vs. Hybrid Cypher Retrieval**


In [12]:
# Helper function to extract ref text
def getChunkDetails(chunkdetails: list):
    chunkdetails = sorted(chunkdetails, key=lambda x:x["score"], reverse=True)
    driver = GraphDatabase.driver(
        uri = uri,
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    
    list_text = []
    
    for chunk in chunkdetails:
        id = chunk["id"]
        score = chunk["score"]
        cypher = "MATCH (n:Chunk) WHERE n.id = $id Return n"    
        data = session.run(cypher,id=id).data()
        text = data[0]["n"]["text"]
        fileName = data[0]["n"]["fileName"]
        list_text.append((fileName,text,score,id))
    return list_text


In [13]:
from neo4j import GraphDatabase

# Helper function to extract ref text
def getChunkDetails(chunkdetails: list):
    chunkdetails = sorted(chunkdetails, key=lambda x:x["score"], reverse=True)
    driver = GraphDatabase.driver(
        uri = uri,
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    
    list_text = []
    
    for chunk in chunkdetails:
        id = chunk["id"]
        score = chunk["score"]
        cypher = "MATCH (n:Chunk) WHERE n.id = $id Return n"    
        data = session.run(cypher,id=id).data()
        text = data[0]["n"]["text"]
        fileName = data[0]["n"]["fileName"]
        list_text.append((fileName,text,score,id))
    return list_text

# Print out reponses of different retrieval methods
print("************************************************")
print(f"This is Graph-only response \n")
print(graph_response['response'])


print("************************************************")
print(f"This is Vector-only response s\n")
print(vector_content)
print("Document Retrieval Scores:")
print(vector_result["chunkdetails"])

print()
print("************************************************")
print(f"This is Hybrid (Vector + Keyword Search) response \t processing time: {predict_time:.3f} s\n")
print(hybrid_content)
print("Document Retrieval Scores:")
print(hybrid_result["chunkdetails"])

print("************************************************")
print(f"This is HybridCypherRetriever Response \n")
print(hybridcypher_response.answer)
print("Document Retrieval Scores:")
print(hybridcypher_chunkdetails[0])

print("************************************************")


# Extract chunks of text retrieved via different 
vector_chunk_context = []
hybrid_chunk_context = []
hybridcypher_chunk_context = []

# OPTIONAL: Utilize getChunkDetails to list ref
# for fileName,context,score,id in getChunkDetails(vector_result["chunkdetails"]):
#     print(f"{fileName} \t score:{score}")
#     print(f"{context} \n")
#     vector_chunk_context.append(context)

# for fileName,context,score,id in getChunkDetails(hybrid_result["chunkdetails"]):
#     print(f"{fileName} \t score:{score}")
#     print(f"{context} \n")
#     hybrid_chunk_context.append(context)

print("************************************************")
for fileName,context,score,id in getChunkDetails(vector_result["chunkdetails"]):
    print(f"{fileName} \t score:{score}")
    print(f"{context} \n")
    vector_chunk_context.append(context)

print("************************************************")
for fileName,context,score,id in getChunkDetails(hybrid_result["chunkdetails"]):
    print(f"{fileName} \t score:{score}")
    print(f"{context} \n")
    hybrid_chunk_context.append(context)

print("************************************************")
for fileName,context,score,id in getChunkDetails(hybridcypher_chunkdetails[0]):
    print(f"{fileName} \t score:{score}")
    print(f"{context} \n")
    hybridcypher_chunk_context.append(context)




************************************************
This is Graph-only response 

I don't know the answer. 

************************************************
This is Vector-only response s

The Circle of Willis is a ring-shaped structure at the base of the brain that ensures continuous blood flow to the brain. It is formed by the following arteries:

* **Internal carotid arteries:** These arteries branch from the common carotid arteries and enter the skull through the carotid canal.
* **Anterior cerebral arteries:** These arteries arise from the internal carotid arteries and supply blood to the frontal lobes and parts of the parietal lobes.
* **Anterior communicating artery:** This small artery connects the two anterior cerebral arteries.
* **Posterior cerebral arteries:** These arteries arise from the basilar artery and supply blood to the occipital lobes and parts of the temporal lobes.
* **Posterior communicating arteries:** These arteries connect the internal carotid arteries to the p

> NOTE: Thresholds for similarity score (i.e default is 0.5) for document retrieval to ensure they are relevant to the query overall
However, in addition to general relevance scores, the Neo4J llm graph builder checks if specific terms or entities directly related to the query are present in the retrieved document. If a document scores highly but doesn't contain key terms or specific entities tied to the query, a fallback response may be triggered.

In [14]:
print(vector_chunk_context)
print(hybrid_chunk_context)
print(hybridcypher_chunk_context)


['of the basilar artery all become the\xa0circle of Willis, a confluence of arter‐ ies that can maintain perfusion of the brain even if narrowing or a block‐ age limits flow through one part (Figure\xa01). Watch this\xa0animation\xa0to see how blood flows to the brain and passes through the circle of Willis before being distributed through the cerebrum. The circle of Willis is a specialized arrange‐ ment of arteries that ensure constant perfusion of the cerebrum even in the event of a blockage of one of the arteries in the circle. The animation shows the normal direction of flow through the cir‐ cle of Willis to the middle cerebral artery. Where would the blood come from if there were a blockage just posterior to the middle cerebral artery on the left? Venous Return After passing through the CNS, blood returns to the circulation through a series of\xa0dural sinuses\xa0and veins (Figure\xa02). The\xa0superior sagittal si‐ n', 'Figure\xa01.\xa0Circle of Willis. The blood supply to the br

In [15]:
import pandas as pd

data = (vector_result["chunkdetails"],hybrid_result["chunkdetails"],hybridcypher_chunkdetails[0])


df = pd.DataFrame(data)
df

,0,1,2,3
0,{'id': 'df99f8192bf2961d93f8fa02eb32337088dbef...,{'id': 'f73ae203cb7d3238826460d93b4dcbef24801c...,{'id': '8eee3b357209bc21f61d633156b9d00b829ddc...,{'id': '24564789b83b665072530bcf37943ede49aa12...
1,{'id': 'df99f8192bf2961d93f8fa02eb32337088dbef...,{'id': 'f73ae203cb7d3238826460d93b4dcbef24801c...,{'id': '8eee3b357209bc21f61d633156b9d00b829ddc...,{'id': '24564789b83b665072530bcf37943ede49aa12...
2,{'id': 'df99f8192bf2961d93f8fa02eb32337088dbef...,{'id': 'f73ae203cb7d3238826460d93b4dcbef24801c...,{'id': '8eee3b357209bc21f61d633156b9d00b829ddc...,None


In [16]:
# RAGAS (default OpenAI)
from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity,
    answer_correctness
)

# Define metrics to evaluate
metrics=[
        context_precision,
        context_recall,
        answer_similarity,
        faithfulness,
        answer_correctness,
        answer_relevancy,
    ]

# Set ground context and ground truth
# reference_contexts=["circle of Willis: unique anatomical arrangement of blood vessels around the base of the brain that maintains perfusion of blood into the brain even if one component of the structure is blocked or narrowed"]
# reference_answer="Circle of Willis is an unique anatomical arrangement of blood vessels around the base of the brain that maintains perfusion of blood into the brain even if one component of the structure is blocked or narrowed"

rubric = {
        "accuracy": "Correct",
        "completeness": "High",
        "fluency":"Excellent"
    }    

# Set ground context and ground_truth (i.e what are all the )
reference_contexts=["Branches off the left and right vertebral arteries merge into the anterior spinal artery supplying the anterior aspect of the spinal cord, found along the anterior median fissure. The two vertebral arteries then merge into the basilar artery, which gives rise to branches to the brain stem and cerebellum. The left and right internal carotid arteries and branches of the basilar artery all become the circle of Willis, a confluence of arter‐ies that can maintain perfusion of the brain even if narrowing or a block‐age limits flow through one part (Figure 1).", "The circle of Willis is a specialized arrange‐ ment of arteries that ensure constant perfusion of the cerebrum even in the event of a blockage of one of the arteries in the circle. The animation shows the normal direction of flow through the cir‐cle of Willis to the middle cerebral artery. Where would the blood come from if there were a blockage just posterior to the middle cerebral artery on the left?"]
reference_answer="internal carotid arteries, vertebral arties, basilar artery"

# Vector Response
sample_vector_retrieval = SingleTurnSample(
    user_input=question,
    retrieved_contexts=vector_chunk_context,
    reference_contexts=reference_contexts,
    response=vector_content,
    reference=reference_answer,
    rubric=rubric
)

# Hybrid (vector+keyword) Response 
sample_hybrid_retrieval = SingleTurnSample(
    user_input=question,
    retrieved_contexts=hybrid_chunk_context,
    reference_contexts=reference_contexts,
    response=hybrid_content,
    reference=reference_answer,
    rubric=rubric
)

# HybridCypher Response only
sample_hybridCypher_retrieval = SingleTurnSample(
    user_input=question,
    retrieved_contexts=hybridcypher_chunk_context,
    reference_contexts=reference_contexts,
    response=hybridcypher_response.answer,
    reference=reference_answer,
    rubric=rubric
)


ds = EvaluationDataset([sample_vector_retrieval, sample_hybrid_retrieval, sample_hybridCypher_retrieval])

result = evaluate(
    dataset=ds,
    metrics = metrics
)

result = result.to_pandas()

Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

## RAGAS Metrics 
**Contextual_Precision**: Measures quality of retrieved results. A ratio of the number of relevant chunks to the total number of k chunks. Utilizes an LLM to determine if retrieved chunks of text are relevant, given a reference context

$\text{Context Precision @ K} = \frac{\sum_{k=1}^{K} (\text{Precision@K} \times v_k)}{\text{Total number of relevant items in the top K results}}$

**Contextual_Recall**: Measures completeness of retrieved results. Computed using user_input, reference (ground truth) answer, and retrieved_contexts with values range between 0 and 1 with higher values indicating better performance. This metric uses reference (ground truth) answer as a proxy to reference_contexts which also makes it easier to use as annotating reference context verus the traditional Non-LLM methods.

$\text{Context Recall} = \frac{|\text{Ground Truth claims that can be attributed to context}|}{|\text{Number of claims in GT}|}$

**Semantic Similarity**: Evaluation based on reference ground truth and answer. This evaluation utilizes a cross-encoder model to calculate the encoder (utilizes cosine similarity) 

$
\text{Cosine Similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$


**Answer Relevancy**: Measures how pertinent the generated question is to the given question. This is computed by generated a number of artifical questions based on the answer and measuring the similarity between the original question and those artificial questions.

The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness.

Answer correctness encompasses two critical aspects: semantic similarity between the generated answer and the ground truth, as well as factual similarity. These aspects are combined using a weighted scheme to formulate the answer correctness score. Users also have the option to employ a 'threshold' value to round the resulting score to binary, if desired.


$\text{Answer Relevancy} = \frac{1}{N}\sum_{i=1}^{N}cos(E_g,E_o)$

where:

 - $E_g$ is the embedding of the generated question.
 - $E_o$ is the embedding of the original question.
 - $N$ is the number of generated questions, which is 3 default.

**Answer correctness**: Measures how accurate the generated answer is relative to a "golden" answer that is deemed to be the correct answer. It is based on weighted sum of factual consistency (F1 Score) and the semantic similarity between the ground-truth answer and the generated response.

$\text{Answer Correctness} = w_1 \times \text{F1-score} + w_2 \times \text{Semantic Similarity}$

where:
- $\text{F1-score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$
- $\text{semantic similarity} = \text{cosine similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$



**Faithfulness**: Measures the factual consistency of the generated answer against given context. The answer is regraded as faithful if all the claims made in the answer can be inferred form the given context. To calculate this, a set of claims from the generated answer is first identified. Then each of these claims is cross-checked with the given context to determine if it can be inferred from the context. Uses an LLM-as-a-judge approach.

$Faithfulness = \frac{|\text{Number of claims in the generated answer that can be inferred from the given answer}|}{|\text{Total Number of claims in generated answer}|}$

***NOTE***: default llm is OpenAI. Requires LLM wrapper to utilize other langchain wrappers


 

In [17]:
result

,user_input,retrieved_contexts,reference_contexts,response,reference,rubric,context_precision,context_recall,semantic_similarity,faithfulness,answer_correctness,answer_relevancy
0,What are all the arteries in the Circle of Willis,[of the basilar artery all become the circle o...,[Branches off the left and right vertebral art...,The Circle of Willis is a ring-shaped structur...,"internal carotid arteries, vertebral arties, b...","{'accuracy': 'Correct', 'completeness': 'High'...",0.0,1.0,0.861038,0.437500,0.871509,0.910007
1,What are all the arteries in the Circle of Willis,[of the basilar artery all become the circle o...,[Branches off the left and right vertebral art...,The Circle of Willis is a network of arteries ...,"internal carotid arteries, vertebral arties, b...","{'accuracy': 'Correct', 'completeness': 'High'...",0.0,1.0,0.870456,0.666667,0.717614,0.920072
2,What are all the arteries in the Circle of Willis,[of the basilar artery all become the circle o...,[Branches off the left and right vertebral art...,The arteries that contribute to the Circle of ...,"internal carotid arteries, vertebral arties, b...","{'accuracy': 'Correct', 'completeness': 'High'...",0.0,1.0,0.893706,0.333333,0.973427,0.958501


In [18]:
metrics_plotted = ['semantic_similarity','faithfulness','answer_correctness','answer_relevancy']

result[metrics_plotted]

,semantic_similarity,faithfulness,answer_correctness,answer_relevancy
0,0.861038,0.437500,0.871509,0.910007
1,0.870456,0.666667,0.717614,0.920072
2,0.893706,0.333333,0.973427,0.958501


## Multiple QA and Retrieval Methods

In [19]:
# Initiate df
df = pd.DataFrame()

### Vector Retrieval

In [20]:
# Import necessary libraries
import pandas as pd
from src.QA_integration_new import (
    format_documents,
    get_rag_chain,
    get_sources_and_chunks,
    get_total_tokens,
    create_document_retriever_chain,
    retrieve_documents
)
from src.llm import get_llm
from src.shared.constants import VECTOR_SEARCH_QUERY, CHAT_SEARCH_KWARG_SCORE_THRESHOLD
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_core.messages import HumanMessage
from ragas import SingleTurnSample

# Set model and load ground truth QA pairs
model = "gemini-1.5-flash-001"
qa_df = pd.read_csv("graphrag_qa_groundtruth.csv")

# Initialize variables
ragas_sample_list = []
llm, model_name = get_llm(model)
score_threshold = CHAT_SEARCH_KWARG_SCORE_THRESHOLD

# Define evaluation rubric
rubric = {
    "accuracy": "Correct",
    "completeness": "High",
    "fluency": "Excellent"
}

# Create vector retriever
neo_db = Neo4jVector.from_existing_index(
    embedding=EMBEDDING_FUNCTION,
    index_name="vector",
    retrieval_query=VECTOR_SEARCH_QUERY,
    graph=graph
)
retriever = neo_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": score_threshold}
)
doc_retriever = create_document_retriever_chain(llm, retriever)

# Iterate through each question-answer pair
for index, row in qa_df.iterrows():
    # Create messages and retrieve documents
    messages = [HumanMessage(content=row["question"])]
    docs = retrieve_documents(doc_retriever, messages)

    if docs:
        formatted_docs, sources = format_documents(docs, model)
        rag_chain = get_rag_chain(llm=llm)
        ai_response = rag_chain.invoke({
            "messages": messages,
            "context": formatted_docs,
            "input": row["question"]  # Use row["question"] instead of undefined variable
        })
        result = get_sources_and_chunks(sources, docs)
        content = ai_response.content
        total_tokens = get_total_tokens(ai_response, llm)
    else:
        content = "I couldn't find any relevant documents to answer your question."
        result = {"sources": [], "chunkdetails": []}
        total_tokens = 0
    print(row["question"])
    print(content)
    # Gather retrieved context details
    retrieved_contexts = [context for _, context, _, _ in getChunkDetails(result["chunkdetails"])]

    # Create RAGAS SingleTurnSample
    sample = SingleTurnSample(
        user_input=str(row["question"]),
        retrieved_contexts=retrieved_contexts,
        reference_contexts=[row["reference_contexts"]],
        response=content,
        reference=str(row["reference_answer"]),
        rubric=rubric
    )

    # Append sample to the list
    ragas_sample_list.append(sample)

# Evaluate the dataset and merge results
ds = EvaluationDataset(ragas_sample_list)
result = evaluate(dataset=ds, metrics=metrics)
result = result.to_pandas()
result["retrieval_type"] = "vector"

# Combine results with existing DataFrame
df = pd.concat([df, result], ignore_index=True)


What is the Circle of Willis?
I'm sorry, but the provided context does not contain information about the Circle of Willis. 

What are all the arteries in the Circle of Willis?
The Circle of Willis is a ring-shaped structure of arteries at the base of the brain. It is formed by the following arteries:

* **Internal carotid arteries:** These arteries branch from the common carotid arteries and enter the skull through the carotid canal.
* **Anterior cerebral arteries:** These arteries arise from the internal carotid arteries and supply the medial and anterior portions of the frontal and parietal lobes.
* **Anterior communicating artery:** This small artery connects the two anterior cerebral arteries.
* **Posterior cerebral arteries:** These arteries arise from the basilar artery and supply the posterior portions of the brain, including the occipital lobes and the temporal lobes.
* **Posterior communicating arteries:** These arteries connect the internal carotid arteries to the posterior c

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

### Hybrid Retrieval

In [21]:
# Import necessary libraries
import pandas as pd
from src.QA_integration_new import (
    format_documents,
    get_rag_chain,
    get_sources_and_chunks,
    get_total_tokens,
    create_document_retriever_chain,
    retrieve_documents
)
from src.llm import get_llm
from src.shared.constants import VECTOR_SEARCH_QUERY, CHAT_SEARCH_KWARG_SCORE_THRESHOLD
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_core.messages import HumanMessage
from ragas import SingleTurnSample

# Set model and load ground truth QA pairs
model = "gemini-1.5-flash-001"
qa_df = pd.read_csv("graphrag_qa_groundtruth.csv")

# Initialize variables
ragas_sample_list = []
llm, model_name = get_llm(model)
score_threshold = CHAT_SEARCH_KWARG_SCORE_THRESHOLD

# Define evaluation rubric
rubric = {
    "accuracy": "Correct",
    "completeness": "High",
    "fluency": "Excellent"
}

# Create vector retriever
neo_db = Neo4jVector.from_existing_graph(
                embedding=EMBEDDING_FUNCTION,
                index_name=index_name,
                retrieval_query=retrieval_query,
                graph=graph,
                search_type="hybrid",
                node_label="Chunk",
                embedding_node_property="embedding",
                text_node_properties=["text"],
                keyword_index_name=keyword_index
                )


retriever = neo_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": score_threshold}
)
doc_retriever = create_document_retriever_chain(llm, retriever)

# Iterate through each question-answer pair
for index, row in qa_df.iterrows():
    # Create messages and retrieve documents
    messages = [HumanMessage(content=row["question"])]
    docs = retrieve_documents(doc_retriever, messages)

    if docs:
        formatted_docs, sources = format_documents(docs, model)
        rag_chain = get_rag_chain(llm=llm)
        ai_response = rag_chain.invoke({
            "messages": messages,
            "context": formatted_docs,
            "input": row["question"]  # Use row["question"] instead of undefined variable
        })
        result = get_sources_and_chunks(sources, docs)
        content = ai_response.content
        total_tokens = get_total_tokens(ai_response, llm)
    else:
        content = "I couldn't find any relevant documents to answer your question."
        result = {"sources": [], "chunkdetails": []}
        total_tokens = 0
    print(row["question"])
    print(content)
    # Gather retrieved context details
    retrieved_contexts = [context for _, context, _, _ in getChunkDetails(result["chunkdetails"])]

    # Create RAGAS SingleTurnSample
    sample = SingleTurnSample(
        user_input=str(row["question"]),
        retrieved_contexts=retrieved_contexts,
        reference_contexts=[row["reference_contexts"]],
        response=content,
        reference=str(row["reference_answer"]),
        rubric=rubric
    )

    # Append sample to the list
    ragas_sample_list.append(sample)

# Evaluate the dataset and merge results
ds = EvaluationDataset(ragas_sample_list)
result = evaluate(dataset=ds, metrics=metrics)
result = result.to_pandas()
result["retrieval_type"] = "hybrid(v+keyword)"

# Combine results with existing DataFrame
df = pd.concat([df, result], ignore_index=True)


What is the Circle of Willis?
The Circle of Willis is a specialized arrangement of arteries at the base of the brain that ensures constant blood flow to the cerebrum, even if there is a blockage in one of the arteries. It's a confluence of arteries that can maintain perfusion of the brain even if narrowing or a blockage limits flow through one part. 



What are all the arteries in the Circle of Willis?
The Circle of Willis is a ring-shaped structure of arteries at the base of the brain. It is formed by the following arteries:

* **Internal carotid arteries:** These arteries enter the skull through the carotid canal and branch into the anterior cerebral artery and the middle cerebral artery.
* **Anterior cerebral arteries:** These arteries supply blood to the frontal lobes and parts of the parietal lobes.
* **Middle cerebral arteries:** These arteries supply blood to the lateral surfaces of the cerebral hemispheres.
* **Posterior cerebral arteries:** These arteries supply blood to the occipital lobes and parts of the temporal lobes.
* **Posterior communicating arteries:** These arteries connect the internal carotid arteries to the posterior cerebral arteries.
* **Anterior communicating artery:** This artery connects the two anterior cerebral arteries.
* **Basilar artery:** This artery is formed by the merging of the two vertebral arte

Why is there a barrier between the central nervous system and the blood supply?
The barrier between the central nervous system (CNS) and the blood supply, known as the blood-brain barrier, exists to protect the delicate brain and spinal cord from harmful substances in the bloodstream. This barrier is crucial because the CNS is highly sensitive to toxins, pathogens, and fluctuations in blood composition. 

The blood-brain barrier acts as a selective filter, allowing essential nutrients and oxygen to pass through while blocking potentially harmful substances. This protection ensures the optimal functioning of the CNS and prevents damage that could lead to neurological disorders. 



How is the pia mater connected with the central nervous system?
The pia mater is directly adjacent to the surface of the central nervous system (CNS). It's a thin fibrous membrane that closely follows the convolutions of the gyri and sulci in the cerebral cortex, as well as other grooves and indentations. This close adherence means the pia mater essentially envelops the CNS, providing a delicate and protective covering. 



How many ventricles exist and how are they connected?
There are four ventricles within the brain: two lateral ventricles, the third ventricle, and the fourth ventricle. 

* The **lateral ventricles** are located deep within the cerebrum and are connected to the **third ventricle** by two openings called the **interventricular foramina**. 
* The **third ventricle** is situated between the left and right sides of the diencephalon and opens into the **cerebral aqueduct**, which passes through the midbrain. 
* The **cerebral aqueduct** then opens into the **fourth ventricle**, which is located between the cerebellum and the pons and upper medulla. 



What happens when there is not enough oxygen to the brain?
When there is not enough oxygen to the brain, it can lead to a condition called **hypoxia**. This can cause a range of problems, from mild cognitive impairment to severe brain damage and even death. 

The brain requires a constant supply of oxygen to function properly. Without oxygen, brain cells begin to die. The severity of the damage depends on how long the brain is deprived of oxygen. 

Here are some of the effects of hypoxia:

* **Confusion and disorientation:**  The brain may not be able to process information correctly, leading to confusion and difficulty thinking clearly.
* **Dizziness and headaches:**  These symptoms can occur due to the brain's inability to function properly.
* **Seizures:**  In severe cases, hypoxia can trigger seizures.
* **Coma:**  If the brain is deprived of oxygen for a long time, it can lead to a coma.
* **Brain damage:**  Prolonged hypoxia can cause permanent brain damage, leading to long-term 

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

### Hybrid Cypher Retrieval

In [22]:
# Now create loads of questions and answers and get RAG metrics

# Question - Answer Ground Truth Pairs with reference
qa_df = pd.read_csv("graphrag_qa_groundtruth.csv")

# Initialize EvaluationDataset()
ragas_sample_list = []

llm,model_name = get_llm(model)

# Hybrid Cypher
mode ="graph + vector + fulltext"

# under QA_RAG with Hybrid Search
retrieval_query = VECTOR_GRAPH_SEARCH_QUERY.format(no_of_entites=VECTOR_GRAPH_SEARCH_ENTITY_LIMIT)

# Create retriever
index_name = "vector"
keyword_index = "keyword"

retriever = HybridCypherRetriever(
    driver=driver,
    vector_index_name=index_name,
    fulltext_index_name=keyword_index,
    retrieval_query=retrieval_query,
    embedder=EMBEDDING_FUNCTION,
)

llm = VertexAILLM(model_name="gemini-1.5-flash-001")
rag = GraphRAG(retriever=retriever, llm=llm)

# Iterate through all questions and answers (ground truth)
for index, row in qa_df.iterrows():
    retriever_result = retriever.search(query_text=row["question"], top_k=4)
    hybridcypher_response = rag.search(query_text=row["question"],retriever_config={"top_k":4})
    hybridcypher_chunkdetails = []
    for m in retriever_result.items:
        hybridcypher_chunkdetails.append(m.metadata["chunkdetails"])
    
    retrieved_contexts = []
    # getChunkDetails
    for fileName,context,score,id in getChunkDetails(hybridcypher_chunkdetails[0]):
        retrieved_contexts.append(context)
    
    # Create RAGAS SingleTurnSample
    sample= SingleTurnSample(
        user_input=str(row["question"]),
        retrieved_contexts=retrieved_contexts,
        reference_contexts=[row["reference_contexts"]],
        response=hybridcypher_response.answer,
        reference=str(row["reference_answer"]),
        rubric=rubric
    )
    print(row["question"])
    print(hybridcypher_response.answer)
    ragas_sample_list.append(sample)
    

ds = EvaluationDataset(ragas_sample_list)
result = evaluate(
    dataset=ds,
    metrics = metrics
)
result = result.to_pandas()
result["retrieval_type"] = "hybridCypher"
df = pd.concat([df,result], ignore_index=True)


What is the Circle of Willis?
The Circle of Willis is a confluence of arteries that can maintain perfusion of the brain even if narrowing or a blockage limits flow through one part. 



What are all the arteries in the Circle of Willis?
The Circle of Willis is formed by the confluence of the following arteries: 

* **Internal carotid arteries:** These arteries enter the cranium through the carotid canal and contribute to the circle. 
* **Vertebral arteries:** These arteries pass through the neck region and enter the cranium through the foramen magnum. They merge to form the basilar artery.
* **Basilar artery:** This artery is formed by the merging of the two vertebral arteries. It gives rise to branches to the brainstem and cerebellum and contributes to the Circle of Willis. 



Why is there a barrier between the central nervous system and the blood supply?
The blood-brain barrier exists to protect the central nervous system (CNS) from harmful substances that might be circulating in the bloodstream. This barrier is critical because the CNS is highly sensitive and vulnerable to toxins, pathogens, and other potentially damaging elements. 



How is the pia mater connected with the central nervous system?
The pia mater is a thin fibrous membrane that directly adjoins the surface of the CNS. It follows the convolutions of the gyri and sulci in the cerebral cortex and fits into other grooves and indentations.  



How many ventricles exist and how are they connected?
There are four ventricles within the brain:

* **Two lateral ventricles:** These are deep within the cerebrum and are connected to the third ventricle by two openings called the interventricular foramina.
* **Third ventricle:** This is the space between the left and right sides of the diencephalon and opens into the cerebral aqueduct.
* **Cerebral aqueduct:** This passage connects the third ventricle to the fourth ventricle through the midbrain.
* **Fourth ventricle:** This is the space between the cerebellum and the pons and upper medulla. 



What happens when there is not enough oxygen to the brain?
Without a steady supply of oxygen, the nervous tissue in the brain cannot keep up its extensive electrical activity. 



Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

In [23]:
# Final Dataframe
desired_col = ['semantic_similarity','faithfulness','answer_correctness','answer_relevancy','retrieval_type']
df[desired_col]
pd.set_option('display.max_colwidth',None)
df[['user_input','response','retrieval_type','faithfulness','answer_correctness','answer_relevancy']].sort_values(by=['user_input','retrieval_type'])

,user_input,response,retrieval_type,faithfulness,answer_correctness,answer_relevancy
9,How is the pia mater connected with the central nervous system?,"The pia mater is directly adjacent to the surface of the central nervous system (CNS). It's a thin fibrous membrane that closely follows the convolutions of the gyri and sulci in the cerebral cortex, as well as other grooves and indentations. This close adherence means the pia mater essentially envelops the CNS, providing a delicate and protective covering. \n",hybrid(v+keyword),1.000000,0.920533,0.970918
15,How is the pia mater connected with the central nervous system?,The pia mater is a thin fibrous membrane that directly adjoins the surface of the CNS. It follows the convolutions of the gyri and sulci in the cerebral cortex and fits into other grooves and indentations. \n,hybridCypher,1.000000,0.971699,0.957396
3,How is the pia mater connected with the central nervous system?,"The pia mater is a thin, delicate membrane that directly adheres to the surface of the central nervous system (CNS). It's like a thin, protective layer that follows the contours of the brain and spinal cord, extending into every convolution and groove. This close connection allows the pia mater to provide a gentle covering and support for the delicate tissues of the CNS. \n",vector,0.714286,0.835625,0.945187
10,How many ventricles exist and how are they connected?,"There are four ventricles within the brain: two lateral ventricles, the third ventricle, and the fourth ventricle. \n\n* The **lateral ventricles** are located deep within the cerebrum and are connected to the **third ventricle** by two openings called the **interventricular foramina**. \n* The **third ventricle** is situated between the left and right sides of the diencephalon and opens into the **cerebral aqueduct**, which passes through the midbrain. \n* The **cerebral aqueduct** then opens into the **fourth ventricle**, which is located between the cerebellum and the pons and upper medulla. \n",hybrid(v+keyword),1.000000,0.585312,0.924431
16,How many ventricles exist and how are they connected?,There are four ventricles within the brain:\n\n* **Two lateral ventricles:** These are deep within the cerebrum and are connected to the third ventricle by two openings called the interventricular foramina.\n* **Third ventricle:** This is the space between the left and right sides of the diencephalon and opens into the cerebral aqueduct.\n* **Cerebral aqueduct:** This passage connects the third ventricle to the fourth ventricle through the midbrain.\n* **Fourth ventricle:** This is the space between the cerebellum and the pons and upper medulla. \n,hybridCypher,1.000000,0.778360,0.924983
4,How many ventricles exist and how are they connected?,"There are four ventricles in the brain: two lateral ventricles, the third ventricle, and the fourth ventricle. \n\n* The **lateral ventricles** are located deep within the cerebrum and are connected to the **third ventricle** by two openings called the **interventricular foramina**. \n* The **third ventricle** is situated between the left and right sides of the diencephalon and opens into the **cerebral aqueduct**, which passes through the midbrain. \n* The **cerebral aqueduct** then opens into the **fourth ventricle**, which is located between the cerebellum and the pons and upper medulla. \n",vector,1.000000,0.614052,0.924931
7,What are all the arteries in the Circle of Willis?,The Circle of Willis is a ring-shaped structure of arteries at the base of the brain. It is formed by the following arteries:\n\n* **Internal carotid arteries:** These arteries enter the skull through the carotid canal and branch into the anterior cerebral artery and the middle cerebral artery.\n* **Anterior cerebral arteries:** These arteries supply blood to the frontal lobes and parts of the parietal lobes.\n* **Middle cerebral arteries:** These arteries supply blood to the lateral surfaces of the cerebral hemispheres.\n* **Poste

In [24]:

# Calculate the average semantic similarity and faithfulness based on retrieval_type
average_metrics = df.groupby('retrieval_type').agg({
    'semantic_similarity': 'mean',
    'faithfulness': 'mean',
    'answer_correctness':'mean',
    'answer_relevancy':'mean'
}).reset_index()

# Rename columns for clarity
average_metrics.columns = ['retrieval_type', 'avg_semantic_similarity', 'avg_faithfulness','avg_correctness','avg_relevancy']

# Define the custom order
custom_order = ['vector', 'hybrid(v+keyword)', 'hybridCypher']

# Create a categorical type with the custom order
average_metrics['retrieval_type'] = pd.Categorical(average_metrics['retrieval_type'], categories=custom_order, ordered=True)

# Sort the DataFrame by the custom order
average_metrics_sorted = average_metrics.sort_values(by='retrieval_type').reset_index(drop=True)

# Round each numeric value to 3 decimal places
average_metrics_sorted.iloc[:, 1:] = average_metrics_sorted.iloc[:, 1:].round(3)

# Display the sorted and rounded DataFrame
average_metrics_sorted

,retrieval_type,avg_semantic_similarity,avg_faithfulness,avg_correctness,avg_relevancy
0,vector,0.878,0.815,0.566,0.776
1,hybrid(v+keyword),0.900,0.856,0.766,0.929
2,hybridCypher,0.924,0.940,0.711,0.938


In [25]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

desired_col = ['semantic_similarity','faithfulness','answer_correctness','answer_relevancy','retrieval_type']

# Create a subplot grid
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=("Faithfulness vs Correctness",
                                    "Relevancy vs Correctness",
                                    "Faithfulness vs Semantic Similarity",
                                    "Correctness vs Semantic Similarity"))

# Plot 1
fig1 = px.scatter(df, 
                  y='faithfulness', 
                  x='answer_correctness', 
                  color='retrieval_type', 
                #   size='answer_relevancy',
                  hover_name=df.index,
                  color_discrete_sequence=px.colors.qualitative.Set1)
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

# Plot 2
fig2 = px.scatter(df,
                  y='answer_relevancy', 
                  x='answer_correctness', 
                  color='retrieval_type',
                  hover_name=df.index,
                  color_discrete_sequence=px.colors.qualitative.Set1)
for trace in fig2.data:
    fig.add_trace(trace.update(showlegend=False), row=1, col=2)

# Plot 3
fig3 = px.scatter(df, 
                  y='faithfulness',
                  x='semantic_similarity',                  
                  color='retrieval_type',
                  hover_name=df.index,
                  color_discrete_sequence=px.colors.qualitative.Set1)
for trace in fig3.data:
    fig.add_trace(trace.update(showlegend=False), row=2, col=1)

# Plot 4
fig4 = px.scatter(df, 
                  y='answer_correctness',
                  x='semantic_similarity', 
                  color='retrieval_type',
                  hover_name=df.index,
                  color_discrete_sequence=px.colors.qualitative.Set1)
for trace in fig4.data:
    fig.add_trace(trace.update(showlegend=False), row=2, col=2)

# Set axis limits and titles for each subplot automatically
axes_info = [
    ("Faithfulness","Correctness"),
    ("Relevancy", "Correctness"),
    ("Faithfulness", "Semantic Similarity"),
    ("Correctness","Semantic Similarity")   
]

for i in range(2):  # Rows
    for j in range(2):  # Columns
        y_axis, x_axis = axes_info[i * 2 + j]
        fig.update_xaxes(title_text=x_axis.replace('_', ' ').capitalize(), 
                         range=[0, 1.05], dtick=0.1, gridwidth=1, row=i + 1, col=j + 1)
        fig.update_yaxes(title_text=y_axis.replace('_', ' ').capitalize(), 
                         range=[0, 1.05], dtick=0.1, gridwidth=1, row=i + 1, col=j + 1)

# Show the plot
fig.update_layout(height=800, width=800, 
                  legend=dict(
                    orientation="h",  # Horizontal legend
                    yanchor="bottom",
                    y=0.475,  # Position just below the title
                    xanchor="center",
                    x=0.5  # Center the legend
                    ),
                  title_text="GraphRAG w/ RAGAS Metrics")
fig.show()


In [26]:
# Visualize Graph
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

# TODO: Chunk ids should not be hardcoded.  
cypher = "MATCH (n:Document)<-[r]->(c:Chunk)<-[s]->(e) WHERE c.id in ['df99f8192bf2961d93f8fa02eb32337088dbeffb', 'f73ae203cb7d3238826460d93b4dcbef24801c62','3a73ea9f64943113b439136645a4e244a6e37935','8eee3b357209bc21f61d633156b9d00b829ddc73']  RETURN n,r,c,s,e LIMIT 1000"

showGraph(cypher)

GraphWidget(layout=Layout(height='670px', width='100%'))

## **Conclusion**

**Graph Search** 
  - **Use Cases**: Good for inherent relationships and connections between nodes in the graph database. Relationship-Focused. 
  - However, the graph response is subject to how well the LLM (i.e Gemini) translate the query into a cypher statement since the graph retrieval for this Neo4j llm-graph-builder repo requires the node and label to match cypher query in order to extract the information under the description attribute of the retrieved node.

**Vector Search** 
  - **Use Cases**: Suitable for NLP tasks like retrieving documents or chunks relevant to a user's query based on context or topic rather than specific keywords
Uses embeddings to capture the semantic meaning of nodes or documents based on similarity (i.e cosine, euclidian). 
  - **Embedding Based**: Represents entities or text as vectors in a high-dimensional space. The similarity between vectors is computed using distance metrics like cosine similarity
  - **Semantic Search**: Focuses on the meaning of the text rather than exact term matching. Allowing it to retrieve relevant information even if exact keywords aren't present.
   - **Example**: Finding documents related to "cerebral arteries" even if the desired term "anterior inferior cerebellar artery" is not mentioned explicitly
  - **Speed and Scalability**: Effective for large datasets where relationships may not be explicitly defined but where semantic meaning is crucial
  - However, it can assume the knowledge graph has additional information beyond the ingested data, when the aggregated embeddings of retrieved documents is translated into human readable response text.

**Hybrid (Vector + Keyword) Search**
  - **Vector Search**: This utilizes embeddings (numerical representations of words or phrases) to capture semantic meaning. It excels at finding relevant content based on similarity rather than exact matches. Use cases include:
    - Finding contextually relevant documents.
    - Identifying similar concepts or entities.
  - **Keyword Search**: This traditional approach matches exact words or phrases within the text. It's efficient for retrieving specific information quickly. Use cases include:
    - Fetching documents containing specific terms.
    - Querying for precise data points.
  - **Hybrid Search**: This combines both methods, allowing the system to consider both semantic relevance and exact matches. It can improve recall and precision by:
    - Prioritizing results based on semantic similarity while also ensuring relevant keyword matches are included.
    - Offering a more nuanced understanding of user queries that may be ambiguous or context-dependent.
  
**Hybrid Cypher Retrieval**: 
  - Combines graph search on top of hybrid (vector + keyword) search methodologies, leveraging the strengths of each approach to enhance retrieval more accurate to the ingested knowledge graph.
  - Incorporates both the relationship-based filtering of graph search on top of semantic similarity assessment of vector search and exact similarity of full-text search via hybrid retrieval.
  - Allows users to retrieve results based on exact relationships (from graph search) while also considering the semantic relevance from vector search
  - **Use Cases**: Best suited for complex queries where both the structural relationships and semantic content are important, such as in knowledge graphs or when navigating large datasets with interrelated concepts
  - **Example**: Finding not only documents about "anterior inferior cerebellar artery" but also documents that are semantically similar while considering the relationships between entities in the graph, like clinical guidelines or related anatomical structures.